<a href="https://colab.research.google.com/github/laurence-lin/Kaggle_competition/blob/master/final_project_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import lightgbm as lgb
import sklearn
import cv2

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc

pd.set_option('display.max_columns' , 500)
pd.set_option('display.width', 300)

print('Library imported.')


Library imported.


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

2

In [ ]:
from google.colab import files

uploaded = files.upload() # upload kaggle.json
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/ 
!kaggle competitions download -c competitive-data-science-predict-future-sales
!unzip \*.zip

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
item_categories.csv: Skipping, found more recently modified local copy (use --force to force download)
sales_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
items.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
shops.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  sales_train.csv.zip
replace sales_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sales_train.csv         

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  items.csv.zip
  infl

In [ ]:

item_cat = pd.read_csv('item_categories.csv')
item = pd.read_csv('items.csv')
sale_train = pd.read_csv('sales_train.csv')
shops = pd.read_csv('shops.csv')

test = pd.read_csv('test.csv')

sample_submission = pd.read_csv('sample_submission.csv')

print('Files imported.')


Files imported.


In [ ]:
print('Items: ')
print(item.head(), '\n')

print('Item category:')
print(item_cat.head(), '\n')

print('Shops data:')
print(shops.head(), '\n')

print('Sales train_data')
print(sale_train.head(), '\n')




Items: 
                                           item_name  item_id  item_category_id
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0                40
1  !ABBYY FineReader 12 Professional Edition Full...        1                76
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2                40
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3                40
4        ***КОРОБКА (СТЕКЛО)                       D        4                40 

Item category:
        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4 

Shops data:
                        shop_name  shop_id
0   !Якутск Орджоникидзе, 56 фран        0
1   !Якутск ТЦ "Центральный" фран        1
2                Адыгея ТЦ "Мега"        2
3  Балашиха ТРК "Октябрь-Киномир"      

###I assume that since I have shop_id and item_category_id, shop name and item_category name is unnecessary. 
###Thus, when I combine the dataset, I ignore item_category and shop dataframe.

In [ ]:
train = sale_train.merge(item, how = 'left', on = 'item_id')
test = test.merge(item, how = 'left', on = 'item_id')
print('Train data:')
print(train.shape)
print(train.head())

print('Test data:')
print(test.shape)
print(test.head())

Train data:
(2935849, 8)
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day                                 item_name  item_category_id
0  02.01.2013               0       59    22154      999.00           1.0                         ЯВЛЕНИЕ 2012 (BD)                37
1  03.01.2013               0       25     2552      899.00           1.0  DEEP PURPLE  The House Of Blue Light  LP                58
2  05.01.2013               0       25     2552      899.00          -1.0  DEEP PURPLE  The House Of Blue Light  LP                58
3  06.01.2013               0       25     2554     1709.05           1.0  DEEP PURPLE  Who Do You Think We Are  LP                58
4  15.01.2013               0       25     2555     1099.00           1.0   DEEP PURPLE 30 Very Best Of 2CD (Фирм.)                56
Test data:
(214200, 5)
   ID  shop_id  item_id                                          item_name  item_category_id
0   0        5     5037                     NHL 15 [PS3

In [ ]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34
print(test.head())

   ID  shop_id  item_id                                          item_name  item_category_id  month  year  date_block_num
0   0        5     5037                     NHL 15 [PS3, русские субтитры]                19     11  2015              34
1   1        5     5320                    ONE DIRECTION  Made In The A.M.                55     11  2015              34
2   2        5     5233  Need for Speed Rivals (Essentials) [PS3, русск...                19     11  2015              34
3   3        5     5232  Need for Speed Rivals (Classics) [Xbox 360, ру...                23     11  2015              34
4   4        5     5268               Need for Speed [PS4, русская версия]                20     11  2015              34


In [ ]:
train['date'] = pd.to_datetime(train['date'])
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
print(train.info())
print(train.head())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column            Dtype         
---  ------            -----         
 0   date              datetime64[ns]
 1   date_block_num    int64         
 2   shop_id           int64         
 3   item_id           int64         
 4   item_price        float64       
 5   item_cnt_day      float64       
 6   item_name         object        
 7   item_category_id  int64         
 8   month             int64         
 9   year              int64         
dtypes: datetime64[ns](1), float64(2), int64(6), object(1)
memory usage: 246.4+ MB
None
        date  date_block_num  shop_id  item_id  item_price  item_cnt_day                                 item_name  item_category_id  month  year
0 2013-02-01               0       59    22154      999.00           1.0                         ЯВЛЕНИЕ 2012 (BD)                37      2  2013
1 2013-03-01               0       25     2552     

In [ ]:
train.isnull().sum()

date                0
date_block_num      0
shop_id             0
item_id             0
item_price          0
item_cnt_day        0
item_name           0
item_category_id    0
month               0
year                0
dtype: int64

In [ ]:
train.loc[train.index[train['item_cnt_day'] < 0], 'item_cnt_day'] = 0
len(train[train['item_cnt_day'] < 0])

0

In [ ]:
# Sum up different days value into monthly
train['cnt_monthly'] = train.groupby(['year', 'month', 'shop_id', 'item_id'])['item_cnt_day'].transform(lambda x: x.sum())
print(train.head())

        date  date_block_num  shop_id  item_id  item_price  item_cnt_day                                 item_name  item_category_id  month  year  cnt_monthly
0 2013-02-01               0       59    22154      999.00           1.0                         ЯВЛЕНИЕ 2012 (BD)                37      2  2013          1.0
1 2013-03-01               0       25     2552      899.00           1.0  DEEP PURPLE  The House Of Blue Light  LP                58      3  2013          1.0
2 2013-05-01               0       25     2552      899.00           0.0  DEEP PURPLE  The House Of Blue Light  LP                58      5  2013          0.0
3 2013-06-01               0       25     2554     1709.05           1.0  DEEP PURPLE  Who Do You Think We Are  LP                58      6  2013          1.0
4 2013-01-15               0       25     2555     1099.00           1.0   DEEP PURPLE 30 Very Best Of 2CD (Фирм.)                56      1  2013          1.0


In [ ]:
from sklearn.model_selection import train_test_split

features = ['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_category_id', 'month', 'year']
y = train['cnt_monthly']
x = train[features]

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.2)

print('Train validate set splited.')


Train validate set splited.


In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(x_train.head())
print(y_train.head())

(2348679, 7)
(587170, 7)
         date_block_num  shop_id  item_id  item_price  item_category_id  month  year
2838144              32       39     3896       349.0                55      9  2015
85308                 0       54    11447       149.0                40      1  2013
2445698              25       53    17717      2200.0                79      2  2015
2092525              21       31    13707       899.0                69      4  2014
2813703              31        3     6090       499.5                23      8  2015
2838144     1.0
85308       2.0
2445698    33.0
2092525     1.0
2813703     1.0
Name: cnt_monthly, dtype: float64


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# scaling

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2)
'''
params = {
    'num_leaves':[50, 80, 150, 250],
    'min_data_in_leaf':[3, 5, 10, 20],
    'learning_rate':0.001,
    'num_iterations':500,
    'device_type':'gpu'
}



grid.fit(x_train, y_train)

'''
params = {
    'max_depth':[10, 50, 80],
    'min_samples_leaf':[5, 10, 20],
    'n_estimators':[100, 200, 300],
}

regressor = RandomForestRegressor()
cv_search = RandomizedSearchCV(regressor,
          n_iter = 13,
          param_distributions = params,
          scoring = 'r2',
          cv = 5,
          verbose = 10,
          n_jobs = -1,
          random_state = 99
          )
cv_search.fit(x_train, y_train)




ImportError: ignored

In [ ]:
!pip install parse_version

ERROR: Could not find a version that satisfies the requirement parse_version (from versions: none)
ERROR: No matching distribution found for parse_version


In [ ]:


#sampling


sub_samples = 10000
sample = train.sample(sub_samples, random_state = 9)

x_sample = sample[features]
y_sample = sample['cnt_monthly']

# scaling
scaler = MinMaxScaler()
x_sample = scaler.fit_transform(x_sample)
x_train, x_valid, y_train, y_valid = train_test_split(x_sample, y_sample, test_size = 0.2)

hidden_layer = [50, 100, 100, 50]
model = MLPRegressor(hidden_layer, 
           alpha = 0.0001, 
           learning_rate = 'adaptive', 
           learning_rate_init = 0.0001,
           verbose = True,
           max_iter = 100000
           )

model.fit(x_train, y_train)
y_pred = model.predict(x_train)
in_score = r2_score(y_train, y_pred)
print('In sample R2 score:', in_score)

y_pred = model.predict(x_valid)
out_score = r2_score(y_valid, y_pred)
print('OOF R2 score:', out_score)

